<a href="https://colab.research.google.com/github/crusader2000/ml_training/blob/master/tensorflow/Tensorflow_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Audiobooks fromm 365 data science

##Loading the csv file

In [0]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('../Audiobooks_data.csv',delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

##Balance the dataset

In [0]:
num_one_targets = int(sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i]==0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)

##Standardize the inputs

In [0]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

##Shuffle the data

In [0]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

##Split the dataset into train, validation and test

In [0]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

##Save the three datasets in *.npz

In [0]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

So that it is easy to reuse we are going to load the data again

##Import the relevant libraries

In [0]:
import numpy as np
import tensorflow as tf

##Data

In [0]:
npz=np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz=np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz=np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

##Model

In [47]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),                             
                            #  tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
                             ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size = 100

max_epochs = 100

#We are using callbacks for early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,train_targets,batch_size=batch_size,epochs=max_epochs,validation_data=(validation_inputs,validation_targets),verbose=2,callbacks=[early_stopping])

Epoch 1/100
36/36 - 0s - loss: 0.5918 - accuracy: 0.7348 - val_loss: 0.4417 - val_accuracy: 0.8949
Epoch 2/100
36/36 - 0s - loss: 0.3916 - accuracy: 0.8662 - val_loss: 0.3075 - val_accuracy: 0.9128
Epoch 3/100
36/36 - 0s - loss: 0.3190 - accuracy: 0.8807 - val_loss: 0.2726 - val_accuracy: 0.9217
Epoch 4/100
36/36 - 0s - loss: 0.2987 - accuracy: 0.8871 - val_loss: 0.2523 - val_accuracy: 0.9195
Epoch 5/100
36/36 - 0s - loss: 0.2866 - accuracy: 0.8905 - val_loss: 0.2409 - val_accuracy: 0.9217
Epoch 6/100
36/36 - 0s - loss: 0.2770 - accuracy: 0.8944 - val_loss: 0.2337 - val_accuracy: 0.9239
Epoch 7/100
36/36 - 0s - loss: 0.2721 - accuracy: 0.8961 - val_loss: 0.2260 - val_accuracy: 0.9262
Epoch 8/100
36/36 - 0s - loss: 0.2650 - accuracy: 0.8947 - val_loss: 0.2337 - val_accuracy: 0.9284
Epoch 9/100
36/36 - 0s - loss: 0.2617 - accuracy: 0.9005 - val_loss: 0.2225 - val_accuracy: 0.9284
Epoch 10/100
36/36 - 0s - loss: 0.2579 - accuracy: 0.8980 - val_loss: 0.2147 - val_accuracy: 0.9284
Epoch 11/

##Testing the data

In [48]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)
print("test_loss:  {0:.2f}.test_accuracy:  {1:.2f}.".format(test_loss,test_accuracy*100.))

14/14 [==============================] - 0s 1ms/step - loss: 0.2456 - accuracy: 0.9219
test_loss:  0.25.test_accuracy:  92.19.
